In [1]:
import os
gpu_device = 2    # número identificador del device puede ser: 0, 1, 2, o 3
os.environ["CUDA_VISIBLE_DEVICES"]=str(gpu_device)
from numba import cuda
cuda.select_device(0)

<weakproxy at 0x7f7245d10f40; to 'numba.cuda.cudadrv.driver.Device' at 0x7f72487c5d30>

In [3]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
import torch
import pandas as pd
import random
import math
from sklearn.metrics import classification_report, accuracy_score
from collections import defaultdict

In [4]:
model_name="microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract"

In [5]:
training_args = TrainingArguments(
    output_dir='./results',          # Carpeta para guardar el modelo
    num_train_epochs=5,             # Número de épocas
    per_device_train_batch_size=16,  # Tamaño del batch
    per_device_eval_batch_size=16,   # Tamaño del batch de validación
    learning_rate=0.01, 
    weight_decay=0.05,              # Decaimiento del peso
    logging_dir='./logs',           # Carpeta para los logs
    logging_steps=10,
    evaluation_strategy="epoch",    # Evaluar durante el entrenamiento
    save_strategy="epoch"           # Guardar el modelo al final de cada época
)

training_args_all= TrainingArguments(
    output_dir='./results',          # Carpeta para guardar el modelo
    num_train_epochs=8,             # Número de épocas
    per_device_train_batch_size=16,  # Tamaño del batch
    per_device_eval_batch_size=16,   # Tamaño del batch de validación
    learning_rate=0.00005, 
    weight_decay=0.05,              # Decaimiento del peso
    logging_dir='./logs',           # Carpeta para los logs
    logging_steps=10,
    evaluation_strategy="epoch",    # Evaluar durante el entrenamiento
    save_strategy="epoch"           # Guardar el modelo al final de cada época
)

/home/mcarrilero/pytorch_env/lib/python3.13/site-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/home/mcarrilero/pytorch_env/lib/python3.13/site-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [6]:
%run ../utils/data.ipynb
%run ../utils/BERT-no-info.ipynb

['biopsy report', 'nodal staging ultrasound report', 'normal control or revision report', 'mammography and ultrasound', 'only mammography study', 'only ultrasound study', 'first degree', 'no family history', 'second degree', 'no prosthesis', 'yes prosthesis', 'BI-RADS 0', 'BI-RADS 1', 'BI-RADS 2', 'BI-RADS 3', 'BI-RADS 4A', 'BI-RADS 4B', 'BI-RADS 4C', 'BI-RADS 5', 'BI-RADS 6', 'ACR A', 'ACR B', 'ACR C', 'ACR D', 'unknown density mammo', 'no calcifications', 'yes calcifications', 'no ganglio', 'yes ganglio', 'fibroglandular and fat', 'heterogeneous fibroglandular', 'homogeneous fatty', 'homogeneous fibroglandular', 'unknown density echo', 'no lymph benign', 'yes lymph benign', 'no lymph suspicious', 'yes lymph suspicious', 'no cyst', 'yes cyst', 'no ectasia', 'yes ectasia', 'no nodules', 'yes nodules', 'irregular', 'lobulated', 'oval', 'round', 'unknown shape', 'circumscribed', 'indistinct', 'not circumscribed', 'spiculated', 'unknown margin', 'complex cystic and solid', 'heterogeneous'

In [9]:
# dic={outputs[i]:0 for i in range(len(outputs))}
# # print(targets)
# # print(outputs)
# for key,res in targets.items():
#     print(key)
#     print(res)
#     # print(outputs[res])
#     dic[outputs[res]]+=1
# print(dic)
# print(len(dic))

In [10]:
len(outputs)

64

In [ ]:
predicted, tested=cross_validation(dataset_final,targets)

EPOCH  0
                                                                                             text
529-854-912-20231002-160122_family              Question: does the patient have any family his...
420-237-263-20221121-115155_density_echo        Question: what is the breast density found in ...
192-976-987-20230612-125013.437_ductal_ectasia  Question: does the following breast medical re...
278-662-412-20211220-123944_density_echo        Question: what is the breast density found in ...
519-750-939-20220627-113153.437_tipo            Question: is the following breast medical repo...
...                                                                                           ...
278-662-412-20211220-123944_lymph_benign        Question: does the following breast medical re...
261-477-582-20230602-103817_family              Question: does the patient have any family his...
346-239-757-20230622-171503_nodules_margin      Question: what is the margin of the first nodu...
136-563-658

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Map: 100%|██████████| 303/303 [00:00<00:00, 391.52 examples/s]


Dataset({
    features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 2724
})
Dataset({
    features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 303
})
Label value check:
Train labels unique: [np.int64(0), np.int64(1), np.int64(2), np.int64(3), np.int64(4), np.int64(5), np.int64(6), np.int64(7), np.int64(8), np.int64(9), np.int64(10), np.int64(11), np.int64(12), np.int64(13), np.int64(14), np.int64(15), np.int64(16), np.int64(17), np.int64(18), np.int64(19), np.int64(20), np.int64(21), np.int64(22), np.int64(23), np.int64(24), np.int64(25), np.int64(26), np.int64(27), np.int64(28), np.int64(29), np.int64(30), np.int64(31), np.int64(32), np.int64(33), np.int64(34), np.int64(35), np.int64(36), np.int64(37), np.int64(38), np.int64(39), np.int64(40), np.int64(41), np.int64(42), np.int64(43), np.int64(44), np.int64(45), np.int64(46), np.int64(47), np.int64(48), np.int64(49), np.int64(50), np.int64(51), np.int64(52), np.int64(53), 

Epoch,Training Loss,Validation Loss
1,3.778400,3.655013
2,2.741200,2.355337
3,2.670700,1.805775
4,2.106800,1.636462
5,1.868700,1.437379


Epoch,Training Loss,Validation Loss
1,0.648600,0.566861
2,0.283300,0.277481
3,0.274500,0.302410
4,0.195300,0.169377
5,0.066300,0.274089
6,0.018600,0.210783
7,0.006600,0.233190
8,0.001600,0.226992


...

[61 13 26 30 22 41  7 27 34 36 42  9 39  3  2 13 25 32 24 40  7 27 34 36
 43  9 39  5  2 13 26 30 22 40  7 27 34 36 42  9 39  3  2 18 26 33 24 40
  7 27 35 36 43 56 59 53 45  9 38  3  2  5  0 13 25 32 22 40  7 27 35 36
 43 56 60 53 48 63  9 38  3  2 13 25 32 24 40  7 27 34 37 43 56 60 53 48
 63  9 38  3  2 13 26 30 22 40  7 27 34 36 42  9 39  3  2 14 25 32 24 40
  7 27 35 36 43 56 60 53 48 63  9 38  5  2 13 25 33 22 40  7 27 34 37 43
 58 60 53 48 63  9 38  3  2 15 25 30 22 40  7 27 34 36 43 56 60 49 46  9
 39  3  2  5  2 13 25 30 24 40  7 27 34 36 43 56 60 49 46 63  9 39  5  2
 13 25 33 22 40  7 27 34 36 42  9 38  3  2 13 26 32 21 40  7 27 34 36 42
  9 38  3  2 12 25 32 21 40  7 27 34 36 42  9 38  3  2 13 25 33 22 40  7
 27 34 37 42  9 38  3  2 14 25 30 24 40  7 27 35 36 43 56 60 53 48  9 39
  5  2 13 26 30 23 40  7 27 34 36 43 58 60 49 46 63  9 38  3  2 13 26 32
 22 41  7 27 34 36 42  9 39  3  2  7  9  5  2]
Accuracy: 0.94
Classification Report:
              precision    recall  f1-s

/home/mcarrilero/pytorch_env/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/mcarrilero/pytorch_env/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/mcarrilero/pytorch_env/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/ho

Dataset({
    features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 2724
})
Dataset({
    features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 303
})
Label value check:
Train labels unique: [np.int64(0), np.int64(1), np.int64(2), np.int64(3), np.int64(4), np.int64(5), np.int64(6), np.int64(7), np.int64(8), np.int64(9), np.int64(10), np.int64(11), np.int64(12), np.int64(13), np.int64(14), np.int64(15), np.int64(16), np.int64(17), np.int64(18), np.int64(19), np.int64(20), np.int64(21), np.int64(22), np.int64(23), np.int64(24), np.int64(25), np.int64(26), np.int64(27), np.int64(28), np.int64(29), np.int64(30), np.int64(31), np.int64(32), np.int64(33), np.int64(34), np.int64(35), np.int64(36), np.int64(37), np.int64(38), np.int64(39), np.int64(40), np.int64(41), np.int64(42), np.int64(43), np.int64(44), np.int64(45), np.int64(46), np.int64(47), np.int64(48), np.int64(49), np.int64(50), np.int64(51), np.int64(52), np.int64(53), 

Epoch,Training Loss,Validation Loss
1,4.155800,3.359910
2,2.888700,2.572382
3,2.579700,2.114468
4,2.101000,1.664944
5,1.741000,1.548236


Epoch,Training Loss,Validation Loss
1,0.451300,0.790953
2,0.300900,0.426210
3,0.223100,0.460967
4,0.142600,0.283357


In [17]:
evaluate_per_question(predicted, tested, DICTIONARY)


🔍 Per-question classification reports:


📘 Question: birads
              precision    recall  f1-score   support

   BI-RADS 0       1.00      0.67      0.80         3
   BI-RADS 1       1.00      1.00      1.00        24
   BI-RADS 2       1.00      1.00      1.00       107
   BI-RADS 3       0.95      0.95      0.95        20
  BI-RADS 4A       0.88      0.95      0.91        22
  BI-RADS 4B       1.00      0.75      0.86         4
  BI-RADS 4C       0.50      0.67      0.57         3
   BI-RADS 5       1.00      1.00      1.00         2
   BI-RADS 6       0.00      0.00      0.00         1

    accuracy                           0.97       186
   macro avg       0.81      0.78      0.79       186
weighted avg       0.97      0.97      0.97       186


📘 Question: calcifications_benign
                    precision    recall  f1-score   support

 no calcifications       0.98      0.99      0.98        99
yes calcifications       0.99      0.98      0.98        87

          accurac

/home/mcarrilero/pytorch_env/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/mcarrilero/pytorch_env/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/mcarrilero/pytorch_env/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(resu

In [16]:
evaluate_per_question(predicted, tested, DICTIONARY)


🔍 Per-question classification reports:


📘 Question: birads
              precision    recall  f1-score   support

   BI-RADS 0       1.00      0.67      0.80         3
   BI-RADS 1       1.00      1.00      1.00        24
   BI-RADS 2       1.00      1.00      1.00       107
   BI-RADS 3       0.95      0.95      0.95        20
  BI-RADS 4A       0.88      0.95      0.91        22
  BI-RADS 4B       1.00      0.75      0.86         4
  BI-RADS 4C       0.50      0.67      0.57         3
   BI-RADS 5       1.00      1.00      1.00         2
   BI-RADS 6       0.00      0.00      0.00         1

    accuracy                           0.97       186
   macro avg       0.81      0.78      0.79       186
weighted avg       0.97      0.97      0.97       186


📘 Question: calcifications_benign
                    precision    recall  f1-score   support

 no calcifications       0.98      0.99      0.98        99
yes calcifications       0.99      0.98      0.98        87

          accurac

/home/mcarrilero/pytorch_env/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/mcarrilero/pytorch_env/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/mcarrilero/pytorch_env/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(resu

In [18]:
np.save("predicted_biomed_8epoch_no_info.npy", predicted)
np.save("tested_biomed_8epoch_no_info.npy", tested)

In [ ]:
✅ Accuracy for: tipo
  biopsy report: 0.8000
  nodal staging ultrasound report: 0.0000
  normal control or revision report: 1.0000
🎯 Overall accuracy: 0.9528

✅ Accuracy for: tecnica
  mammography and ultrasound: 0.9867
  only mammography study: 0.0000
  only ultrasound study: 1.0000
🎯 Overall accuracy: 0.9811

✅ Accuracy for: family
  first degree: 0.8889
  no family history: 0.9937
  second degree: 0.7778
🎯 Overall accuracy: 0.9731

✅ Accuracy for: prosthesis
  no prosthesis: 1.0000
  yes prosthesis: 0.9000
🎯 Overall accuracy: 0.9946

✅ Accuracy for: birads
  BI-RADS 0: 0.0000
  BI-RADS 1: 1.0000
  BI-RADS 2: 1.0000
  BI-RADS 3: 1.0000
  BI-RADS 4A: 1.0000
  BI-RADS 4B: 0.5000
  BI-RADS 4C: 0.0000
  BI-RADS 5: 0.0000
  BI-RADS 6: 0.0000
🎯 Overall accuracy: 0.9409

✅ Accuracy for: density_mammo
  ACR A: 1.0000
  ACR B: 0.9512
  ACR C: 0.9872
  ACR D: 0.9286
  unknown density mammo: 0.9722
🎯 Overall accuracy: 0.9677

✅ Accuracy for: calcifications_benign
  no calcifications: 1.0000
  yes calcifications: 0.9770
🎯 Overall accuracy: 0.9892

✅ Accuracy for: ganglio_mamo
  no ganglio: 0.9944
  yes ganglio: 0.4286
🎯 Overall accuracy: 0.9731

✅ Accuracy for: density_echo
  fibroglandular and fat: 0.8333
  heterogeneous fibroglandular: 0.9750
  homogeneous fatty: 0.2500
  homogeneous fibroglandular: 0.9649
  unknown density echo: 0.9091
🎯 Overall accuracy: 0.9355

✅ Accuracy for: lymph_benign
  no lymph benign: 0.9673
  yes lymph benign: 0.8485
🎯 Overall accuracy: 0.9462

✅ Accuracy for: lymph_suspicious
  no lymph suspicious: 0.9942
  yes lymph suspicious: 0.6667
🎯 Overall accuracy: 0.9677

✅ Accuracy for: simple_cyst
  no cyst: 0.9703
  yes cyst: 0.9882
🎯 Overall accuracy: 0.9785

✅ Accuracy for: ductal_ectasia
  no ectasia: 1.0000
  yes ectasia: 0.9375
🎯 Overall accuracy: 0.9946

✅ Accuracy for: nodules_echo
  no nodules: 0.9231
  yes nodules: 0.9512
🎯 Overall accuracy: 0.9355

✅ Accuracy for: nodules_shape
  irregular: 0.0000
  lobulated: 0.8462
  oval: 0.9600
  round: 0.0000
  unknown shape: 0.9250
🎯 Overall accuracy: 0.8780

✅ Accuracy for: nodules_margin
  circumscribed: 0.9394
  indistinct: 0.3333
  not circumscribed: 0.0000
  spiculated: 0.0000
  unknown margin: 0.9524
🎯 Overall accuracy: 0.8780

✅ Accuracy for: nodules_echogenicity
  complex cystic and solid: 0.0000
  heterogeneous: 0.0000
  hypoechoic: 1.0000
  isoechoic: 0.0000
  unknown echogenicity: 0.9310
🎯 Overall accuracy: 0.9146

✅ Accuracy for: nodules_known
  no known: 0.8421
  yes known: 0.8182
🎯 Overall accuracy: 0.8293

✅ Accuracy for: nodules_stable
  grown stable: 0.0000
  shrunk stable: 0.0000
  unknown stable: 0.0000
  yes stable: 0.9429
🎯 Overall accuracy: 0.7500

In [ ]:
✅ Accuracy for: tipo
  biopsy report: 0.8500
  nodal staging ultrasound report: 0.3333
  normal control or revision report: 1.0000
🎯 Overall accuracy: 0.9670

✅ Accuracy for: tecnica
  mammography and ultrasound: 0.9933
  only mammography study: 0.0000
  only ultrasound study: 0.9833
🎯 Overall accuracy: 0.9811

✅ Accuracy for: family
  first degree: 0.9444
  no family history: 1.0000
  second degree: 0.8889
🎯 Overall accuracy: 0.9892

✅ Accuracy for: prosthesis
  no prosthesis: 1.0000
  yes prosthesis: 0.9000
🎯 Overall accuracy: 0.9946

✅ Accuracy for: birads
  BI-RADS 0: 0.6667
  BI-RADS 1: 1.0000
  BI-RADS 2: 1.0000
  BI-RADS 3: 1.0000
  BI-RADS 4A: 0.9545
  BI-RADS 4B: 1.0000
  BI-RADS 4C: 0.3333
  BI-RADS 5: 0.0000
  BI-RADS 6: 0.0000
  unknown BI-RADS: [No samples]
🎯 Overall accuracy: 0.9624

✅ Accuracy for: density_mammo
  ACR A: 0.6667
  ACR B: 0.9512
  ACR C: 0.9872
  ACR D: 0.9643
  unknown density mammo: 0.9722
🎯 Overall accuracy: 0.9677

✅ Accuracy for: calcifications_benign
  no calcifications: 0.9899
  yes calcifications: 0.9655
🎯 Overall accuracy: 0.9785

✅ Accuracy for: ganglio_mamo
  no ganglio: 0.9944
  yes ganglio: 0.8571
🎯 Overall accuracy: 0.9892

✅ Accuracy for: density_echo
  fibroglandular and fat: 0.9167
  heterogeneous fibroglandular: 0.9750
  homogeneous fatty: 0.5000
  homogeneous fibroglandular: 0.9825
  unknown density echo: 0.8788
🎯 Overall accuracy: 0.9462

✅ Accuracy for: lymph_benign
  no lymph benign: 0.9804
  yes lymph benign: 0.7879
🎯 Overall accuracy: 0.9462

✅ Accuracy for: lymph_suspicious
  no lymph suspicious: 0.9942
  yes lymph suspicious: 0.6667
🎯 Overall accuracy: 0.9677

✅ Accuracy for: simple_cyst
  no cyst: 0.9802
  yes cyst: 0.9882
🎯 Overall accuracy: 0.9839

✅ Accuracy for: ductal_ectasia
  no ectasia: 1.0000
  yes ectasia: 0.9375
🎯 Overall accuracy: 0.9946

In [12]:
evaluate_per_question(predicted, tested, DICTIONARY)


🔍 Per-question classification reports:


📘 Question: birads
                 precision    recall  f1-score   support

      BI-RADS 0       0.00      0.00      0.00         3
      BI-RADS 1       0.96      1.00      0.98        24
      BI-RADS 2       0.90      0.94      0.92       107
      BI-RADS 3       0.65      0.55      0.59        20
     BI-RADS 4A       0.45      0.64      0.53        22
     BI-RADS 4B       0.00      0.00      0.00         4
     BI-RADS 4C       0.00      0.00      0.00         3
      BI-RADS 5       0.00      0.00      0.00         2
      BI-RADS 6       0.00      0.00      0.00         1
unknown BI-RADS       0.00      0.00      0.00         0

      micro avg       0.81      0.81      0.81       186
      macro avg       0.30      0.31      0.30       186
   weighted avg       0.77      0.81      0.78       186


📘 Question: calcifications_benign
                    precision    recall  f1-score   support

 no calcifications       0.94      0.99   

/home/mcarrilero/pytorch_env/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/mcarrilero/pytorch_env/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/mcarrilero/pytorch_env/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(resul

              precision    recall  f1-score   support

  no ectasia       0.99      1.00      0.99       170
 yes ectasia       1.00      0.88      0.93        16

    accuracy                           0.99       186
   macro avg       0.99      0.94      0.96       186
weighted avg       0.99      0.99      0.99       186


📘 Question: family
                   precision    recall  f1-score   support

     first degree       0.94      0.94      0.94        18
no family history       1.00      1.00      1.00       159
    second degree       0.89      0.89      0.89         9

         accuracy                           0.99       186
        macro avg       0.94      0.94      0.94       186
     weighted avg       0.99      0.99      0.99       186


📘 Question: ganglio_mamo
              precision    recall  f1-score   support

  no ganglio       0.99      0.99      0.99       179
 yes ganglio       0.83      0.71      0.77         7

    accuracy                           0.98    

/home/mcarrilero/pytorch_env/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/mcarrilero/pytorch_env/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/mcarrilero/pytorch_env/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(resu